# Dzień czwarty DWthon - Hack Outside the Box 

*Dzisiaj zaczniemy już trenować modele ML, dla Ciebie to oznacza, że jesteś we właściwym miejsce do praktykowania, zdobywania nowych praktycznych umiejętności (w tym kodowanie, ML... ale coś więcej), inspiracja, poznawania ciekawych ludzi. Co ważne, zaoszczędzanie czasu, bo materiał jest tak poukładany, aby jak najszybciej dać Ci możliwość czerpać garściami praktyczną wiedzę. Ciszę się, że jesteś z nami 🥳* Zaczynamy?

### Celem jest wytrenować modele - zajmiemy się  `machine learning` w działaniu 🤖

Dzisiaj wytrenujemy kilka modeli. Najpierw poznasz drzewa decyzyjne, a następnie zastosujemy `gradient boosting`. Jeśli dopiero zaczynasz, to te nazwy nic Ci nie mówią. Właściwie i nie muszą, najważniejsze dzisiaj jest to, aby udało Ci się wytrenować modele, nawet jeśli jest to dla Ciebie "black-box", czyli nie wiesz, co tam się dzieje w środku, ale wiesz, że udało się wytrenować!

### Gdzie zadawać pytania ❓ 

Jeśli napotkasz trudności podczas wykonywania zadań z tego notebooka, to koniecznie napisz w kanale [dwthon_day4](https://bit.ly/30zgHbn)

Pamiętaj, aby szczegółowo doprecyzować, z czym masz problem. Najlepiej wrzuć screen z kodem swoim lub błędem, który widzisz i napisz, którego zadania dotyczy :)

In [1]:
%%html
<iframe style="height:500px;width:100%" src="https://www.youtube.com/embed/ntJGWqkFINE" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

### Krok po kroku 

Nagrałem dla Ciebie także materiał wideo "krok po kroku". W wideo poniżej znajduje się dokładnie to, co w tym notebooku tylko tłumaczę wszystko, aby ułatwić Ci pracę i zrozumienie zagadnień i zadań :) Obejrzyj, jeśli potrzebujesz takiego dodatkowego wsparcia. 

In [2]:
%%html
<iframe style="height:500px;width:100%" src="https://www.youtube.com/embed/Jxo4Oy_KxfY" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

## Biblioteki

Potrzebujemy `pandas` i jeszcze trochę więcej innych bibliotek. 
* `DecisionTreeClassifier` - do trenowania modelu
* `sklearn.model_selection` walidacja modelu
* `gc` do sprzątania w pamięci RAM

In [3]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, GroupKFold

import gc

## Wczytajmy dane

In [4]:
df = pd.read_hdf("../input/data.h5")

print(df.shape)
df.sample(5)

(820906, 9)


,order_id,customer_id,product_id,quantity,price_unit,price_total,country_id,order_date,is_canceled
802386,51972,3810,120,6,295,1770,0,2010-11-29 12:49:00,False
624701,41135,1186,1281,32,42,1344,0,2010-07-22 19:56:00,False
56201,4048,1226,365,2,95,190,0,2011-02-13 12:03:00,False
136496,9660,1096,236,24,125,3000,1,2011-05-18 10:34:00,False
715748,46954,2230,0,32,254,8128,0,2010-10-14 09:31:00,False


## 🎯 Zmienna docelowa 

Musimy ustalić, co chcemy prognazować. Możemy prognozować różne rzeczy, ML model chętnie nam w tym pomoże. Natomiast musimy jawnie mu pokazać, na czym nam zależy! Pomyśl w ten sposób. *Machine Learning* (ML) jest taką wróżką 🧚, która próbuje spełnić to, na czym nam zależy, ale musimy wiedzieć, czego chcemy :). 

Załóżmy, że na początek, chcemy umieć prognazować, czy klient X należy do segmentu `most_revenue_customer`. W takim razie policzmy `top_customers` (tak, jak to robiliśmy ostatnio).

In [5]:
df_customers = (
    df[ ["price_total", "customer_id"] ]
    .groupby("customer_id")
    .agg("sum")
    .reset_index()
    .sort_values(by="price_total", ascending=False)
    .rename(columns={"price_total": "customer_price_total"})
)


df_customers["cumsum"] = df_customers["customer_price_total"].cumsum()
value_80prc = int(df["price_total"].sum() * 0.8)
df_customers["most_revenue_customer"] = df_customers["cumsum"] < value_80prc


top_customers = set(df_customers[ df_customers["most_revenue_customer"] ]["customer_id"].unique())

del df_customers
gc.collect()

0

Na końcu wywołaliśmy: 

```
del df_customers
gc.collect()
```

To dlatego, aby zwolnić pamięć. Już nie potrzebujemy zmiennej `df_customers`, potrzebowaliśmy jej to po to, aby wyliczyć `top_customers`. Słowa kluczowe `del` zwalnia zasoby, natomiast `gc.collect()` woła "sprzątacz" (ang. `garbage collector`), aby posprzątał jak najszybciej. 

Tak nawiasem mówiąc, fajnny jest ten świat IT - chcesz posprzątać 🗑️, wtedy wołasz `gc.collect()` i już masz 💪.


## Klienci
Teraz mamy nasz oryginalny zbiór danych sprowadzić do pojedyńczych klientów. Innymi słowy jeden wiersz to jeden unikalny klient. W tym celu użyjemy tak jak ostatnio `groupby` po `customer_id`.


Dodatkowo zrobimy bardziej zaawansowaną agregację, ponieważ przyszedł na to już czas:).

Jako przykład: `count_orders=("order_id", lambda x: len(set(x)))`

Ten wpis oznacza, że nowa kolumna będzie miała nazwę `count_orders`. Natomiast aby ta kolumna powstała, to bierzemy oryginalną kolumnę o nazwie `order_id` i używamy do tego funkcję agregującą `lambda x: len(set(x))`. Konkretnie w tym przypadku liczymy liczbę unikalnych zamowień.

##### ☝️Uwaga! Jeśli to, co jest poniżej w `agg` dla Ciebie jest trudne, nie przejmuj się tym! Na ten moment to jest malo istotne, aby dokładnie zrozumieć, jak to pisze się na poziomie kodu. Ważne jest to, że rozumisz, że chcemy dla każdego klienta policzyć, ile ma zamówień albo ile kupił unikalnych produktów itd.

In [6]:
df_customers = (
    df
    .groupby("customer_id")
    .agg(
        count_orders=("order_id", lambda x: len(set(x))),
        count_unq_products=("product_id", lambda x: len(set(x))),
        sum_quantity=("quantity", np.sum),
        sum_price_unit=("price_unit", np.sum),
        sum_price_total=("price_total", np.sum),
        count_unq_countries=("country_id", lambda x: len(set(x))),
        prob_canceled=("is_canceled", np.mean)
    ).reset_index()
)

df_customers["most_revenue_customer"] = df_customers["customer_id"].map(lambda x: x in top_customers)

df_customers

,customer_id,count_orders,count_unq_products,sum_quantity,sum_price_unit,sum_price_total,count_unq_countries,prob_canceled,most_revenue_customer
0,0,159,93,22976,895827,5746671,1,0.010966,True
1,1,40,184,3514,205962,910113,1,0.097727,True
2,2,30,139,8971,95500,1337464,1,0.009050,True
3,3,9,40,1108,18626,226381,1,0.000000,False
4,4,14,11,390,20028,346978,1,0.307692,True
...,...,...,...,...,...,...,...,...,...
5874,5874,2,46,213,15654,48343,1,0.040000,False
5875,5875,1,21,261,4944,30081,1,0.000000,False
5876,5876,1,69,367,33622,110868,1,0.000000,False
5877,5877,1,33,85,7995,13458,1,0.000000,False


Dodatkowo używaliśmy wcześniej wyliczonego `top_customers`, aby powstała nasza zmienna docelowa `most_revenue_customer`.

Sprawdźmy przy okazji (przypomnijmy), jaki jest rozkład.

In [7]:
df_customers["most_revenue_customer"].value_counts(normalize=True)

False    0.774792
True     0.225208
Name: most_revenue_customer, dtype: float64

Mamy 22,5% klilentów należących do `top_customers`. 

Dobra! Czas na trenowanie modelu.

# Szybki słowniczek ML


## Co to jest algorytm (np. drzewa decyzyjne)?
To jest pewna sekwencja kroków, która będzie wykonywana, aby trenować model (i nauczyć go najlepiej).


## Czym jest model?
Jest pewnym "stanem" tego, co udało się znaleźć w danych historycznych.

Przykład przez analogię.

Algorytm - to jest przepis na danie. Model to jest przygotowane dane. 
Przepis jest jeden, ale dań może powstać million i będą różne w zależności od tego, co było podane "na wejściu" (wrzucone do miski).


## Co to oznacza trenować model?

Jest to proces, kiedy model poszukuje zależności (korelacji) w danych historycznych. Wynikiem trenowania jest model, którego potem możemy używać do prognozowania przyszłości (lub w inny sposób, są różne kreatywne podejścia).


## Co to są cechy?
To są właściwości obiektów. W naszym przypadku cechami dla klienta może być: liczba zamówień, liczba produktów itd.

Można powiedzieć, że cechy to są kolumny w naszej tablicy (pomijając zmienną docelową).

## Co to jest zmienna docelowa?
To jest to, co model będzie prognozował. 

Można powiedzieć, że to jest kolumna w naszej tabeli, na której nam zależy.


## Dlaczego zmienna docelowa nie może być w cechach?
Model jest "sprytny". On od razu "złapie", że to jest odpowiedź i będzie ją zwracał. Natomiast jaka jest z tego wartość? Jeśli ma z góry wiadomą odpowiedź. To nie ma sensu przepuszczać tego przez model.

Przykład.

Jeśli my z góry wiemy, że ten klient należy do segmentu `top_customers`, to już to wiemy. Po co nam wtedy model? Druga sprawa, że zwykle nie wiemy, ale znamy inne cechy tego klienta (np. ilu już dokonał zamówień, jakie produkty kupił itd.) i na podstawie tej informacji próbujemy wnioskować, czy będzie należał do segmentu `top_customers` czy też nie.


## Po co trenujemy model?
Zwykle po to, aby prognozować w przyszłość. W naszym przypadku chodzi o bardziej kreatywne podejście i dokładnie to zobaczysz jutro. Dzisiaj skupmy się na tym, aby wytrenować model i to dobrze, wg metryki sukcesu.


## Co to jest metryka sukcesu?
To jest sposób, aby zmierzyć jakość modelu. Pomyśl o tym w ten sposób. Mamy dwa różne modele. Mamy dwa rożne wyniki, w jaki sposób możemy stwierdzić, który model jest lepszy? Metryka sukcesu sprowadza się do tego, że "kompresuje" jakość modelu do pojedynczej liczby. Wtedy dwa różne modele dostaje dwie różne liczby, które można porównać pomiędzy sobą.

Przykładową metryką jest dokładność. Np. model A ma dokładność 95%, model B ma 99%. Wychodzi na to, ze model B jest lepszy. Chociaż co do metryki jest bardzo dużo niuansów, które koniecznie trzeba rozumieć, ale nie wnikajmy w te szczególiki teraz.



### 🤔 Zwróć uwagę, że liczba pytań "dlaczego?" może być znacznie większa. Celem DWthon jest bardziej przeprowadzić Cię od punktu A do punktu B  i pokazać, że też możesz, nawet jeśli czasem bardzo mocno prowadzę Ci rączkę. Jeśli sam proces Ci spodobał się to kolejnym krokiem jest poznać więcej szczegółów, jak działa model, jak działa walidacja i jeszcze sporo ważnych niuansów. Zapraszam Cię na 8 tygodniowy [online kurs praktyczne uczenie maszynowe od podstaw](https://dataworkshop.eu/pl/practical-machine-learning?utm_source=dwthon&utm_medium=notebook&utm_campaign=python&utm_term=day4&utm_content=jupyter). Już  3000+ osób przerobiło go z takich firm jak Google, Orange, Oracle, IBM, Microsoft, Intel, Allegro, Nokia i wiele innych. Tu możesz zobaczyć [opinie absolwentów](https://bit.ly/3vcz3wU).


Wracamy. Przygotujemy teraz cechy.

Poniżej będzie kod, który robi prostą rzeczy, mianowicie zostawia tylko takie kolumne, które są numerczyne albo `boolean` (czyli kolumne, które mają wartości `True` lub `False`).

In [8]:
feats = list(df_customers.select_dtypes([np.number, bool]).columns)

feats

['customer_id',
 'count_orders',
 'count_unq_products',
 'sum_quantity',
 'sum_price_unit',
 'sum_price_total',
 'count_unq_countries',
 'prob_canceled',
 'most_revenue_customer']

Jak widzisz mamy już nazwy cech, ale trafiło do nich `most_revenue_customer`, czyli nasza zmienna docelowa. Należy ją usunąć. Możemy zrobić taką listę cech, którą chemy zignorować. W tej chwili w tej liście będzie tylko jedna kolumna `most_revenue_customer`, ale spokojnie można dodawać więcej.

In [9]:
black_list = ["most_revenue_customer"]
feats = [x for x in feats if x not in black_list]

Został nam minimalny krok do postawienia, aby powstał tak zwany `X` oraz `y`.

- `X` - to jest macierz cech.
- `y` - to jest nasza zmienna docelowa.

In [10]:
X = df_customers[feats].values
y = df_customers["most_revenue_customer"].values

In [13]:
df_customers[feats]

,customer_id,count_orders,count_unq_products,sum_quantity,sum_price_unit,sum_price_total,count_unq_countries,prob_canceled
0,0,159,93,22976,895827,5746671,1,0.010966
1,1,40,184,3514,205962,910113,1,0.097727
2,2,30,139,8971,95500,1337464,1,0.009050
3,3,9,40,1108,18626,226381,1,0.000000
4,4,14,11,390,20028,346978,1,0.307692
...,...,...,...,...,...,...,...,...
5874,5874,2,46,213,15654,48343,1,0.040000
5875,5875,1,21,261,4944,30081,1,0.000000
5876,5876,1,69,367,33622,110868,1,0.000000
5877,5877,1,33,85,7995,13458,1,0.000000


### Trenujemy model

Na początkek użyjemy drzewa decyzjnego. Jak wspomniałem wcześniej, nie musisz wiedzieć z góry, jak działa ten model, potraktuj go jako "black-box", który "jakoś" potrafi się uczyć.

In [14]:
scores = cross_val_score(DecisionTreeClassifier(max_depth=5), X, y, scoring="accuracy", cv=5)
np.mean(scores), np.std(scores)

(0.9996598639455783, 0.0004165798882284457)

Dostaliśmy bardzo dobry wynik: 99% dokładności. Gratuluję! Wlaśnie powstał Twój pierwszy model.

## Zadanie 4.1

Przed chwilą trenowaliśmy model dla `top_customers`. Twoim zadaniem jest wytrenować model `top_products`


### 💡 Podpowiedzi: 

1. Najpierw przygotuj zbiór `top_products`.
2. Następnie przygotuj `df_products`, który będzie zawierał przydatne cechy (zainspiruj się jak to zrobiliśmy dla `df_customers`).
3. Następnie przygotuj `feats`, `X` i `y`.
4. Wytrenuj model (np. `DecisionTreeClassifier`). 
5. Podziel się wynikiem :) 

In [16]:
df_products = (
    df[ ["price_total", "product_id"] ]
    .groupby("product_id")
    .agg("sum")
    .reset_index()
    .sort_values(by="price_total", ascending=False)
    .rename(columns={"price_total": "product_price_total"})
)


df_products["cumsum"] = df_products["product_price_total"].cumsum()
value_80prc = int(df["price_total"].sum() * 0.8)
df_products["most_revenue_product"] = df_products["cumsum"] < value_80prc


top_products = set(df_products[ df_products["most_revenue_product"] ]["product_id"].unique())

del df_products
gc.collect()

23

In [17]:
df_products = (
    df
    .groupby("product_id")
    .agg(
        count_orders=("order_id", lambda x: len(set(x))),
        count_unq_products=("customer_id", lambda x: len(set(x))),
        sum_quantity=("quantity", np.sum),
        sum_price_unit=("price_unit", np.sum),
        sum_price_total=("price_total", np.sum),
        count_unq_countries=("country_id", lambda x: len(set(x))),
        prob_canceled=("is_canceled", np.mean)
    ).reset_index()
)

df_products["most_revenue_product"] = df_products["product_id"].map(lambda x: x in top_products)

df_products

,product_id,count_orders,count_unq_products,sum_quantity,sum_price_unit,sum_price_total,count_unq_countries,prob_canceled,most_revenue_product
0,0,5028,1494,97329,1525206,26097687,22,0.025179,True
1,1,614,293,4638,248166,1650981,6,0.028358,True
2,2,736,320,7160,261086,2267898,11,0.016753,True
3,3,1475,706,16707,742149,5850395,14,0.009699,True
4,4,601,324,3804,552166,2979279,17,0.037994,True
...,...,...,...,...,...,...,...,...,...
3873,4231,1,1,1,465,465,1,0.000000,False
3874,4236,4,4,29,340,2465,1,0.000000,False
3875,4237,2,2,5,590,1475,1,0.000000,False
3876,4238,1,1,200,1942,388400,1,0.000000,False


In [18]:
df_products["most_revenue_product"].value_counts(normalize=True)

False    0.761475
True     0.238525
Name: most_revenue_product, dtype: float64

In [22]:
feats = list(df_products.select_dtypes([np.number, bool]).columns)

# feats

black_list = ["most_revenue_product"]
feats = [x for x in feats if x not in black_list]

X = df_products[feats].values
y = df_products["most_revenue_product"].values

In [23]:
df_products[feats]

,product_id,count_orders,count_unq_products,sum_quantity,sum_price_unit,sum_price_total,count_unq_countries,prob_canceled
0,0,5028,1494,97329,1525206,26097687,22,0.025179
1,1,614,293,4638,248166,1650981,6,0.028358
2,2,736,320,7160,261086,2267898,11,0.016753
3,3,1475,706,16707,742149,5850395,14,0.009699
4,4,601,324,3804,552166,2979279,17,0.037994
...,...,...,...,...,...,...,...,...
3873,4231,1,1,1,465,465,1,0.000000
3874,4236,4,4,29,340,2465,1,0.000000
3875,4237,2,2,5,590,1475,1,0.000000
3876,4238,1,1,200,1942,388400,1,0.000000


In [24]:
scores = cross_val_score(DecisionTreeClassifier(max_depth=5), X, y, scoring="accuracy", cv=5)
np.mean(scores), np.std(scores)


(0.999484203525108, 0.0006317193061419072)

Jak **wykonasz** zadanie, należy zrobić:

1. Zrzut ekranu na którym widać rozwiązania, wtedy dostaniesz bonus :)
2. Wrzucić rozwiązanie na slacku do pokoju **[#dwthon_day4_done](https://bit.ly/30yqIWm)**
3. Dostać bonus za dobrą robotę 💪 i jutro będzie kolejna porcja.

*Uwaga! Jeśli masz problem z jakimś zadaniem, czegoś nie wiesz, to pamiętaj, że możesz zadawać pytania! Na tym polega nauka :) Pytania związane z 2 dniem DWthon zadawaj w kanale [#dwthon_day4](https://bit.ly/30zgHbn)*.

## Dodatkowe zadania


Przechodzimy do dodatkowych zadań i rozważań. Nie musiz tego robić, ale chyba warto :).

## 🧠 Włączmy myślenie krytycznie


Pomyślmy, co tak dokładnie teraz robimy.

Mamy listę naszych klientów w momencie X (ostatni zakup), ale przecież oś czasu jest bardzo ważna. Zbadajmy to na przykładzie.

In [25]:
df["order_date"].min(), df["order_date"].max()

(Timestamp('2009-12-01 07:45:00'), Timestamp('2011-12-09 12:50:00'))

Mamy dane od `2009-12-01 07:45:00` do `2011-12-09 12:50:00`. 

Jak trenujemy model, to do modelu przekazujemy całą historię, jaką mamy na temat klienta. To wszystko powoduje, że model uczy się na sytuacjach, kiedy już o kliencie wiemy "dość dużo". Tak naprawdę może być tak, że już nie trzeba złożonego modelu, aby zobaczyć, że to jest klient z segmentu `top_customer`.

Fajnie było trenować model trochę inaczej i wykrywać jego potencjał jeszcze na samym początku.

Pomyśl o tym w ten sposób.


Mamy historię o kliencie np. na 2 lata. Wiemy, że on jest w segmencie `top_customer`. Natomiast do tego segmentu dołączył się dopiero... powiedzmy po 15 miesiącach. Jeśli przesuniemy w czasie to pomiędzy 0-15 miesiącem nie było jednoznaczności, że ten klient będzie w `top_customer`, po 15 miesiącach już tam jest, więc nawet nie trzeba ML, aby to uznać.


### Co możemy zrobić?
No właśnie to jest dobre pytanie. Zrozumienie osi czasu w ML jest bardzo ważne i z mojego doświadczenia wynika, że właśnie tutaj popełnia się najwięcej błędów w szczególności przez początkujących.


## 🧠 Włączmy myślenie krytycznie
Porozważajmy, jak możemy wykorzystać naszą sytuację. 

1. Wiemy, czym się "skończy" (bo już jest w `top_customers`) lub tego jeszcze nie wiemy, bo ktoś tam będzie w przyszłości (danych tych nie mamy jeszcze).
2. Ale chcemy pokazać modelowi tylko odcinek danych (np. znamy historię sprzed 2 lat, ale modelowi pokazujemy tylko pierwsze 10 miesięcy, 12 miesięcy , 15 miesięcy).


Porozważaj, jak możemy trenować model, aby z jednej strony model dostawał jak najmniej informacji (czyli jak najszybciej był wstanie "rozpoznać" potencjał). Jakie są plusy i minusy (przynajmniej w Twoim odczuciu).


#### ☝️Zwróć uwagę, że to nie chodzi o techniczną "rozkminę", jak to zrobić (jakoś to się napisze lub kogoś można zapytać). Chodzi bardziej, aby przemyśleć to koncepcyjnie, czyli pomyśleć, co chcemy osiągnąć :) 

Na czym nam zależy? Czy model ma nauczyć sie na już bardziej zaawansowanym "stanie" klienta, kiedy historia jest wystarczająco długa (np. 1-2 lata i tak już wiadoma, kto jest kto), czy bardziej wykrywać jak najwcześniej "złoty potencjał". To drugie brzmi jak coś, co ma znacznie wiecej wartości dodanej. Tutaj też Machine Learning może mieć przewagę nad "klasyczną analityką".

## 🤝🗣️ Uruchom kooperację i komunikację


Fajnie byłoby o tym podyskutować :) 


#### ☝️ Podziel się swoimi przemyśleniami na slacku **[#dwthon_day4_ideas](https://bit.ly/3tcKIdn)**. Jeśli masz pomysł, ale nie masz kodu - poproś na slacku o pomoc, aby ktoś pomógł Ci napisać kod. Natomiast jeśli masz kod, to koniecznie podziel się. Dzięki temu ktoś inny będzie w stanie łatwo powtórzyć i podzielić się swoimi przemyśleniami. Dzięki temu każdy zyskuje :).

Uwaga! Aby to było czytelne, proponuję dodawać tagi do wiadmości na samym początku.

Na przykład.
```
#how_to_train_model

Moje wnioski ...
1. ...
2. ...
3. ...
```

Zawartość pisz, jak Ci jest wygodnie, ale zacznijmy od tagu `#how_to_train_model`, dzięki temu łatwiej będzie można filtrować wśród innych wiadomości.

--- 


## Widzimy się jutro!  🙌